In [1]:
!pip install pydicom
!pip install opencv-python
import pandas as pd
import numpy as np
from pathlib import Path
import random
import pickle
import json
from pydicom import dcmread
from pydicom.data import get_testdata_file
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from keras.models import load_model, Model
from keras.callbacks import ModelCheckpoint
from keras.metrics import BinaryAccuracy, Precision, Recall
from sklearn.model_selection import train_test_split
import tensorflow as tf
import time
import cv2
import gc
import plotly.express as px

In [ ]:
home_folder= ''
data_gen_folder = 'data_gen/'

In [2]:
# Run iff executing in Google Colab
from google.colab import drive
drive.mount('/content/drive')
home_folder = '/content/drive/My Drive/Colab Notebooks/rsna-intercranial-aneurysm-detection/'
data_gen_folder = home_folder + 'data_gen_2/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train = pd.read_csv(home_folder + 'data/train.csv')
train_localizers = pd.read_csv(home_folder + 'data/train_localizers.csv')

In [ ]:
train.head()

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
0,1.2.826.0.1.3680043.8.498.10004044428023505108...,64,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.8.498.10004684224894397679...,76,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.2.826.0.1.3680043.8.498.10005158603912009425...,58,Male,CTA,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,1.2.826.0.1.3680043.8.498.10009383108068795488...,71,Male,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.8.498.10012790035410518400...,48,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_localizers.head()

,SeriesInstanceUID,SOPInstanceUID,coordinates,location
0,1.2.826.0.1.3680043.8.498.10005158603912009425...,1.2.826.0.1.3680043.8.498.10775329348174902199...,"{'x': 258.3621186176837, 'y': 261.359900373599}",Other Posterior Circulation
1,1.2.826.0.1.3680043.8.498.10022796280698534221...,1.2.826.0.1.3680043.8.498.53868409774237283281...,"{'x': 194.87253141831238, 'y': 178.32675044883...",Right Middle Cerebral Artery
2,1.2.826.0.1.3680043.8.498.10023411164590664678...,1.2.826.0.1.3680043.8.498.24186535344744886473...,"{'x': 189.23979878597123, 'y': 209.19184886465...",Right Middle Cerebral Artery
3,1.2.826.0.1.3680043.8.498.10030095840917973694...,1.2.826.0.1.3680043.8.498.75217084841854214544...,"{'x': 208.2805049088359, 'y': 229.78962131837307}",Right Infraclinoid Internal Carotid Artery
4,1.2.826.0.1.3680043.8.498.10034081836061566510...,1.2.826.0.1.3680043.8.498.71237104731452368587...,"{'x': 249.86745590416498, 'y': 220.623044646393}",Anterior Communicating Artery


In [ ]:
train_localizers['coordinates_dict'] = train_localizers['coordinates'].str.replace("'", '"').apply(json.loads)
train_localizers['frame'] = train_localizers['coordinates_dict'].apply(lambda d: d['f'] if 'f' in d else -1)
train_df = train_localizers[['SeriesInstanceUID', 'SOPInstanceUID', 'frame']].copy()
train_df['in_brain_with_aneurysm'] = 1
train_df['aneurysm_visible_in_image'] = 1

In [ ]:
train_df.head()

,SeriesInstanceUID,SOPInstanceUID,frame,in_brain_with_aneurysm,aneurysm_visible_in_image
0,1.2.826.0.1.3680043.8.498.10005158603912009425...,1.2.826.0.1.3680043.8.498.10775329348174902199...,-1,1,1
1,1.2.826.0.1.3680043.8.498.10022796280698534221...,1.2.826.0.1.3680043.8.498.53868409774237283281...,-1,1,1
2,1.2.826.0.1.3680043.8.498.10023411164590664678...,1.2.826.0.1.3680043.8.498.24186535344744886473...,-1,1,1
3,1.2.826.0.1.3680043.8.498.10030095840917973694...,1.2.826.0.1.3680043.8.498.75217084841854214544...,-1,1,1
4,1.2.826.0.1.3680043.8.498.10034081836061566510...,1.2.826.0.1.3680043.8.498.71237104731452368587...,-1,1,1


# Part 2: Dual Prediction

I am fortunate enough to be collaborating on this project with a number of talented medical professionals. Recently, I held a discussion with my collaborators Niyati Shelat, Yelena Trepetin, and Carlos Puyo. One of the challenges we discussed was that of over 1 million medical images present in the dataset, only about 2,000 contain an aneurysm. There are 4,348 patients, about half of which are patients with an aneurysm. But, even of the patients with an aneurysm, the folder corresponding to their series_id typically contains hundreds of images, and only one or a few of those images actually contain a visible aneurysm. And so the question is, how do we effectively use the remaining images - images from a patient with an aneurysm, but where an aneurysm is not explicitly visible?

Niyati and Carlos hypothesized that even if an aneurysm is not explicitly visible in those images, there may be other signs that a deep learning algorithm could pick up on that indicate there could be an aneurysm elsewhere in the brain. What exactly these indications would be is not necessarily obvious, but aneurysms arise due to a weakening of the walls of blood vessels, and are associated with certain other health conditions, such as certain kidney diseases. This would indicate that there may be more systemic features present in these individuals, which a deep learning algorithm could hypothetically pick up on, even in images with do not contain a visible aneurysm.

So what we arived at was the idea of dual prediction - rather than taking in a single image and predicting a single binary feature - whether it contains an aneurysm - we take in a single image and predict two, **non-mutually exclusive** binary features: whether it contains an aneurysm, **and** whether it is in a brain that contains an aneurysm (anywhere in the brain). (Of course, any image that contains a visible aneurysm will also be in a brain that contains an aneurysm - but not all images from a brain that contains an aneurysm will themselves contain a visible aneurysm).

This solves the key issue of what to do about the large percentage of images from patients with an aneurysm that do not contain a visible aneurysm. These images will be labeled with a `0` for the `aneurysm_visible_in_image` feature, but a `1` for the `in_brain_with_aneurysm` feature. Meanwhile, images from patients without an aneurysm will be labeled with a `0` for both features, while images with a visible aneurysm will be labeled with a `1` for both features (and, of course, no images will be labeled with a `1` for `aneurysm_visible_with_image` and a `0` for `in_brain_with_aneurysm`).

Below, I have modified the image selection code to still follow the same procedure of selecting 5 random images from each folder (including every image with an aneurysm), but now label each image according to 2 binary variables instead of 1; `aneurysm_visible_in_image` and `in_brain_with_aneurysm`.

Later, I plan to use all of the images from each folder for training/validation, but this requires the step of setting up a multi-GPU VM using Google Cloud/Azure/AWS. For my final report for my course, my goal is just to get this new logic up and running using the old sampling procedure; then, during the period after the final project is due but before the competition closes, I will put in the work to modify the setup to use all data and train the model on a multi-GPU setup.

In [ ]:
random.seed(42)
start_time = time.time()
total_images_skipped = 0
for series_id, aneurysm_present in zip(train['SeriesInstanceUID'], train['Aneurysm Present']):
    # Select a random sample of five images from each series
    # Add to train_df with frame = -2 to indicate aneurysm absent
    # Check to ensure the series + image id is not already in train_df first
    series_directory = Path(home_folder + 'series/' + series_id)
    all_images_in_series = [file.stem for file in series_directory.iterdir()]
    frame = -1
    for i in range(0, min(5, len(all_images_in_series))):
        randomly_chosen_image_id = random.choice(all_images_in_series)
        if randomly_chosen_image_id not in train_df.SOPInstanceUID:
            read_image = dcmread(home_folder + 'series/' + series_id + '/' + randomly_chosen_image_id + '.dcm', stop_before_pixels=True)
            if hasattr(read_image, "NumberOfFrames"):
                # (A few images seem to have wonky metadata, just skip those)
                if not isinstance(read_image.NumberOfFrames, int):
                    total_images_skipped += 1
                    continue
                num_frames = int(read_image.NumberOfFrames)
                if num_frames > 1:
                    frame = random.randint(0, num_frames - 1)
            train_df.loc[len(train_df)] = [series_id, randomly_chosen_image_id, frame, aneurysm_present, 0]

print(f'Image selection algorithm complete. Elapsed time: {(time.time() - start_time)} seconds. Total images skipped: {total_images_skipped}')

Image selection algorithm complete. Elapsed time: 59.93915128707886 seconds. Total images skipped: 5


In [ ]:
train_df.sample(20)

,SeriesInstanceUID,SOPInstanceUID,frame,in_brain_with_aneurysm,aneurysm_visible_in_image
22202,1.2.826.0.1.3680043.8.498.97256479550884529885...,1.2.826.0.1.3680043.8.498.11130529916904112748...,-1,1,0
1747,1.2.826.0.1.3680043.8.498.74873401525166495912...,1.2.826.0.1.3680043.8.498.33276051072716875081...,-1,1,1
574,1.2.826.0.1.3680043.8.498.13043233553603781549...,1.2.826.0.1.3680043.8.498.11049814333974884841...,-1,1,1
9850,1.2.826.0.1.3680043.8.498.24345011661898711698...,1.2.826.0.1.3680043.8.498.47734654454041592411...,-1,0,0
13663,1.2.826.0.1.3680043.8.498.46744819661701610682...,1.2.826.0.1.3680043.8.498.47757599722202228945...,-1,0,0
16164,1.2.826.0.1.3680043.8.498.61871174149382284545...,1.2.826.0.1.3680043.8.498.81373163076183434740...,-1,0,0
8914,1.2.826.0.1.3680043.8.498.18695486015446530695...,1.2.826.0.1.3680043.8.498.96193835591015601018...,-1,0,0
16526,1.2.826.0.1.3680043.8.498.64053235207957596184...,1.2.826.0.1.3680043.8.498.66690615945318306687...,-1,1,0
16186,1.2.826.0.1.3680043.8.498.61978777386049223022...,1.2.826.0.1.3680043.8.498.77544587165109240851...,-1,0,0
6640,1.2.826.0.1.3680043.8.498.12657866377259426932...,1.2.826.0.1.3680043.8.498.56027375338481820433...,-1,1,0


In [ ]:
train_df_file_path = data_gen_folder + 'train_df.csv'

In [ ]:
train_df.to_csv(train_df_file_path, index=False)

In [ ]:
train_df_loaded = pd.read_csv(train_df_file_path)

Next, we want to split the data into a train set and a validation set. We use sklearn's `train_test_split`, with a test_size of 0.15 to put 15% of the data into the validation set and leave the other 85% in the training set. Very importantly, we use `stratify = training_image_labels` to ensure equivalent proportions of positive and negative images in both the train and validation sets. Positive images (i.e. images with aneurysms) still make up only ~10% of the selected training images, and with such imbalanced classes it is crucial that the validation set matches the overall proportions.

In [ ]:
train_df_X = list(zip(train_df_loaded.SeriesInstanceUID, train_df.SOPInstanceUID, train_df.frame))
train_df_y = train_df[['in_brain_with_aneurysm', 'aneurysm_visible_in_image']]

X_train_paths, X_val_paths, y_train, y_val = train_test_split(
    train_df_X, train_df_y, test_size = 0.15, random_state=42, stratify = train_df_y
)

In [ ]:
len(X_train_paths), len(X_val_paths), len(y_train), len(y_val)

(19295, 3406, 19295, 3406)

#### Preprocessing image data

Here comes the trickiest part. We have a wide variety of different kinds of medical images, with a wide variety of different dimensions. Many but not all of the images are square. To feed this into a convolutional network, it would be best to ensure a standardized format (that is, equivalent x and y dimensions for all images).

The images are in the DICOM medical imaging format, so we use `pydicom` to read the image data.

We then need to convert all images into standard dimensions, e.g. 512x512, by some combination of scaling/padding/cropping.

We also should rescale the pixel values to be between zero and one. (The images are all greyscale)

Finally, we need to convert each processed image into a 2d numpy array or tensor of `float16`s. `float16` is the smallest floating point format which will greatly reduce the total size of the images. All of these 2d arrays/tensors will then need to be combined into a single large array to then feed as input to our convolutional neural network in the next phase.

(Additionally, we strongly consider data augmentation, where we create rotated/distorted/blurred versions of our training data. This would likely be very helpful in this case because there are so few example images that contain aneurysms (just over 2,000). However, due to my limited computational resources, I will skip this step for the time being. Potentially, I will revisit this later when trying to improve performance of the network.)

The preprocessing phase is implemented below:

In [3]:
processed_image_dim = 512
processed_image_dimensions = (processed_image_dim, processed_image_dim)

# MRIs done by GE

def preprocess_image(image_info_tuple):
    series_id, image_id, frame = image_info_tuple
    image_path = home_folder + 'series/' + series_id + '/' + image_id + '.dcm'

    # Steps:
    # Read DICOM image and convert to numpy pixel array
    pixel_array = dcmread(image_path).pixel_array.astype(np.float32)
    if frame == -1:
        assert len(pixel_array.shape) == 2, f'frame == -1 but pixel array shape is {pixel_array.shape}'
    else:
        assert len(pixel_array.shape) == 3, f'frame == {frame} but pixel array shape is {pixel_array.shape}'
        pixel_array = pixel_array[frame]

    # - Rescale pixel values to range [0, 1]
    pixel_min, pixel_max = pixel_array.min(), pixel_array.max()
    if pixel_max == pixel_min:
        pixel_array_scaled = np.zeros_like(pixel_array, dtype=np.float32)
    else:
        pixel_array_scaled = (pixel_array - pixel_min) / (pixel_max - pixel_min)

    # - Pad/scale to 512x512
    try:
        x_dim, y_dim = pixel_array_scaled.shape
    except Exception as e:
        print(f'Exception due to unexpected shape {pixel_array_scaled.shape}. Image path: {image_path}')

    if x_dim != y_dim:
        max_dim = max(x_dim, y_dim)

        pad_x = max_dim - x_dim
        pad_left = pad_x // 2
        pad_right = pad_x - pad_left

        pad_y = max_dim - y_dim
        pad_top = pad_y // 2
        pad_bottom = pad_y - pad_top

        previous_shape = pixel_array_scaled.shape
        pixel_array_scaled = np.pad(
            pixel_array_scaled,
            pad_width=((pad_left, pad_right), (pad_top, pad_bottom)),
            mode='constant',
            constant_values=0
        )
        x_dim, y_dim = pixel_array_scaled.shape
        assert x_dim == y_dim, f'x_dim should equal y_dim. x_dim == {x_dim}, y_dim == {y_dim}. Previous shape: {previous_shape}'

    if x_dim != processed_image_dim:
        if x_dim > processed_image_dim:
            pixel_array_scaled = cv2.resize(pixel_array_scaled, processed_image_dimensions, cv2.INTER_AREA)
        if x_dim < processed_image_dim:
            pixel_array_scaled = cv2.resize(pixel_array_scaled, processed_image_dimensions, cv2.INTER_CUBIC)

    # Convert to float16 to reduce memory usage
    return pixel_array_scaled.astype(np.float16)

In [ ]:
start_time = time.time()
X_val = np.asarray([preprocess_image(image_info_tuple) for image_info_tuple in X_val_paths])
print(f'Completed preprocessing {len(X_val)} validation images. Elapsed time: {time.time() - start_time} seconds.')

Completed preprocessing 3406 validation images. Elapsed time: 69.2662992477417 seconds.


In [4]:
X_val_file_path = data_gen_folder + 'X_val.pkl'
X_train_file_path = data_gen_folder + 'X_train.pkl'

In [ ]:
with open(X_val_file_path, 'wb') as file:
    pickle.dump(X_val, file)

In [ ]:
start_time = time.time()
X_train = np.asarray([preprocess_image(image_info_tuple) for image_info_tuple in X_train_paths])
print(f'Completed preprocessing {len(X_train)} training images. Elapsed time: {time.time() - start_time} seconds.')

C:\ProgramData\anaconda3\envs\tensorflow\Lib\site-packages\pydicom\pixels\utils.py:222: UserWarning: A value of 'None' for (0028,0008) 'Number of Frames' is invalid, assuming 1 frame
  warn_and_log(


Completed preprocessing 19295 training images. Elapsed time: 367.1058032512665 seconds.


In [ ]:
with open(X_train_file_path, 'wb') as file:
    pickle.dump(X_train, file)

In [5]:
y_val_file_path = data_gen_folder + 'y_val.csv'
y_train_file_path = data_gen_folder + 'y_train.csv'

In [ ]:
y_val.to_csv(y_val_file_path, index=False)
y_train.to_csv(y_train_file_path, index=False)

#### Loading saved images

The preprocessing steps were completed above, and then the training and validation data were saved as `.pkl` and `.txt` files. Below, we load the saved data into `x_train_loaded`, `x_val_loaded`, `y_train_loaded`, and `y_val_loaded`.

In [6]:
y_val_loaded = pd.read_csv(y_val_file_path)
y_train_loaded = pd.read_csv(y_train_file_path)

y_val_loaded = np.array(y_val_loaded)
y_train_loaded = np.array(y_train_loaded)

y_val_in_brain = y_val_loaded[:, 0]
y_val_visible = y_val_loaded[:, 1]
y_train_in_brain = y_train_loaded[:, 0]
y_train_visible = y_train_loaded[:, 1]

In [7]:
with open(X_val_file_path, 'rb') as file:
    X_val_loaded = pickle.load(file)

In [8]:
with open(X_train_file_path, 'rb') as file:
    X_train_loaded = pickle.load(file)

#### Model Architecture

Below, I define the architecture of my convolutional neural network. For this architecture, I started with the sample architecture given by Professor Josh Hug in Lecture 22.2:

In [ ]:
# Do not execute: for demonstration purposes only
model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(filters=32, kernel_size=3, activation="relu"),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=64, kernel_size=3, activation="relu"),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=128, kernel_size=3, activation="relu"),
    Flatten(),
    Dense(10, activation="softmax")
])

Many changes needed to be made to this architecture for this problem, however:

- With input images of `512x512` instead of `28x28`, the input shape needs to be changed to `(512, 512, 1)`
- As this is a binary classification tasks rather than a classification task with 10 classes, the final layer needs to be changed to a dense layer with 1 node and a sigmoid activation function
- Because the images are so much larger, I added in a couple more convolutional layers to reduce the size of the feature space.
- I also added in a couple Dropout layers to hopefully help with overfitting
- Finally, after noticing that the final `Flatten()` layer was generating a huge number of parameters, I switched it out for a `GlobalAveragePooling2D()` layer based on some recommendations I found online.
- I also added `BatchNormalization` layers to some of the later convolutional layers.

My chosen architecture is shown below. Note that this is only my initial architecture and I will try to further tune it later. But for now this seemed like a reasonable starting point.

In [9]:
inputs = Input(shape=(processed_image_dim, processed_image_dim, 1))

x = Conv2D(32, 3, activation="relu")(inputs)
x = MaxPooling2D(2)(x)

x = Conv2D(64, 3, activation="relu")(x)
x = MaxPooling2D(2)(x)

x = Conv2D(128, 3, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(2)(x)
x = Dropout(0.25)(x)

x = Conv2D(256, 3, activation="relu")(x)
x = MaxPooling2D(2)(x)

x = Conv2D(256, 3, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)

out_1 = Dense(1, activation='sigmoid', name='in_brain_with_aneurysm')(x)
out_2 = Dense(1, activation='sigmoid', name='aneurysm_visible_in_image')(x)

model = Model(inputs, [out_1, out_2])

def create_metrics(name_suffix):
    return [
        BinaryAccuracy(threshold=0.3, name='accuracy_.3_' + name_suffix),
        BinaryAccuracy(threshold=0.5, name='accuracy_.5_' + name_suffix),
        BinaryAccuracy(threshold=0.7, name='accuracy_.7_' + name_suffix),
        Precision(thresholds=0.3, name='precision_.3_' + name_suffix),
        Precision(thresholds=0.5, name='precision_.5_' + name_suffix),
        Precision(thresholds=0.7, name='precision_.7_' + name_suffix),
        Recall(thresholds=0.3, name='recall_.3_' + name_suffix),
        Recall(thresholds=0.5, name='recall_.5_' + name_suffix),
        Recall(thresholds=0.7, name='recall_.7_' + name_suffix),
    ]

model.compile(
    optimizer='rmsprop',
    loss={'in_brain_with_aneurysm': 'binary_crossentropy', 'aneurysm_visible_in_image': 'binary_crossentropy'},
    metrics={
        'in_brain_with_aneurysm': create_metrics('in_brain'),
        'aneurysm_visible_in_image': create_metrics('visible'),
    }
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 510, 510,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 255, 255,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 253, 253,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 126, 126,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 124, 124,  │     73,728 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 124, 124,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 124, 124,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 62, 62,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 62, 62,    │          0 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 60, 60,    │    295,168 │ dropout[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 30, 30,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 28, 28,    │    589,824 │ max_pooling2d_3[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │      1,024 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 28, 28,    │          0 │ batch_normalizat… │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ activation_1[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ global_average_p

 Total params: 979,586 (3.74 MB)

 Trainable params: 978,818 (3.73 MB)

 Non-trainable params: 768 (3.00 KB)

In [10]:
training_history_file_path = data_gen_folder + 'training_history.txt'
model_save_file_path = data_gen_folder + 'saved_model.keras'

In [11]:
X_train_loaded = X_train_loaded[..., np.newaxis]
X_val_loaded = X_val_loaded[..., np.newaxis]

In [13]:
val_ds = tf.data.Dataset.from_tensor_slices((
    X_val_loaded,
    {
        'in_brain_with_aneurysm': y_val_in_brain,
        'aneurysm_visible_in_image': y_val_visible,
    }
)).batch(32).prefetch(tf.data.AUTOTUNE)

train_ds = tf.data.Dataset.from_tensor_slices((
    X_train_loaded,
    {
        'in_brain_with_aneurysm': y_train_in_brain,
        'aneurysm_visible_in_image': y_train_visible,
    }
)).batch(32).prefetch(tf.data.AUTOTUNE)

In [14]:
del X_train_loaded
del X_val_loaded
gc.collect()

60

In [15]:
save_weights_callback = ModelCheckpoint(
    filepath=data_gen_folder+'checkpoints/epoch_{epoch:02d}_val_loss_{val_loss:.4f}.weights.h5',
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=True,
    mode='min',
    verbose=1
)

In [16]:
history = model.fit(
    train_ds,
    epochs = 40, # Through experimentation, findings show that model starts overfitting around epoch 25-30.
    validation_data = val_ds,
    callbacks=[save_weights_callback],
)

# Save model architecture and training history
model.save(model_save_file_path)
with open(training_history_file_path, 'w', encoding='utf-8') as file:
    json.dump(history.history, file, indent=4, ensure_ascii=False)

Epoch 1/40
603/603 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - aneurysm_visible_in_image_accuracy_.3_visible: 0.8472 - aneurysm_visible_in_image_accuracy_.5_visible: 0.8829 - aneurysm_visible_in_image_accuracy_.7_visible: 0.8941 - aneurysm_visible_in_image_loss: 0.3595 - aneurysm_visible_in_image_precision_.3_visible: 0.1276 - aneurysm_visible_in_image_precision_.5_visible: 0.1315 - aneurysm_visible_in_image_precision_.7_visible: 0.0491 - aneurysm_visible_in_image_recall_.3_visible: 0.0783 - aneurysm_visible_in_image_recall_.5_visible: 0.0277 - aneurysm_visible_in_image_recall_.7_visible: 0.0019 - in_brain_with_aneurysm_accuracy_.3_in_brain: 0.4930 - in_brain_with_aneurysm_accuracy_.5_in_brain: 0.5226 - in_brain_with_aneurysm_accuracy_.7_in_brain: 0.5285 - in_brain_with_aneurysm_loss: 0.7371 - in_brain_with_aneurysm_precision_.3_in_brain: 0.4859 - in_brain_with_aneurysm_precision_.5_in_brain: 0.5050 - in_brain_with_aneurysm_precision_.7_in_brain: 0.5544 - in_brain_with_aneurysm_recall_.3_in_br

In [17]:
with open(training_history_file_path, 'r', encoding='utf-8') as file:
    history_loaded = json.loads(file.read())

In [18]:
model_loaded = load_model(model_save_file_path)

In [31]:
history_loaded_df = pd.DataFrame(history_loaded).reset_index().rename(columns={'index':'epoch'})
history_loaded_df.head()

,epoch,aneurysm_visible_in_image_accuracy_.3_visible,aneurysm_visible_in_image_accuracy_.5_visible,aneurysm_visible_in_image_accuracy_.7_visible,aneurysm_visible_in_image_loss,aneurysm_visible_in_image_precision_.3_visible,aneurysm_visible_in_image_precision_.5_visible,aneurysm_visible_in_image_precision_.7_visible,aneurysm_visible_in_image_recall_.3_visible,aneurysm_visible_in_image_recall_.5_visible,...,val_in_brain_with_aneurysm_accuracy_.5_in_brain,val_in_brain_with_aneurysm_accuracy_.7_in_brain,val_in_brain_with_aneurysm_loss,val_in_brain_with_aneurysm_precision_.3_in_brain,val_in_brain_with_aneurysm_precision_.5_in_brain,val_in_brain_with_aneurysm_precision_.7_in_brain,val_in_brain_with_aneurysm_recall_.3_in_brain,val_in_brain_with_aneurysm_recall_.5_in_brain,val_in_brain_with_aneurysm_recall_.7_in_brain,val_loss
0,0,0.880954,0.896605,0.899663,0.331527,0.137143,0.130841,0.083333,0.037578,0.007307,...,0.570464,0.515854,0.674851,0.484724,0.584007,0.666667,1.000000,0.393939,0.001212,0.984188
1,1,0.894221,0.899819,0.900441,0.303708,0.229437,0.206897,0.142857,0.027662,0.003132,...,0.574574,0.517616,0.672301,0.509055,0.598820,0.888889,0.936970,0.369091,0.004848,0.964041
2,2,0.889816,0.900492,0.900544,0.293041,0.318339,0.467742,0.285714,0.096033,0.015136,...,0.566941,0.516148,0.678409,0.519422,0.634823,1.000000,0.915758,0.249697,0.001212,0.982887
3,3,0.887536,0.899663,0.900337,0.283204,0.352326,0.435065,0.347826,0.158142,0.034969,...,0.568702,0.516442,0.684177,0.552738,0.662478,1.000000,0.832121,0.223636,0.001818,0.994714
4,4,0.883908,0.899197,0.900648,0.275739,0.349442,0.429952,0.481481,0.196242,0.046451,...,0.585731,0.519378,0.670657,0.537374,0.678092,0.933333,0.875758,0.275758,0.008485,0.959792


In [32]:
baseline_visible_aneurysm_accuracy = 1.0 - (y_val_visible.sum() / len(y_val_visible))
baseline_visible_aneurysm_precision = 1.0
baseline_visible_aneurysm_recall = 0.0

baseline_in_brain_accuracy = 1.0 - (y_val_in_brain.sum() / len(y_val_in_brain))
baseline_in_brain_precision = 1.0
baseline_in_brain_recall = 0.0

history_loaded_df['baseline_visible_accuracy'] = baseline_visible_aneurysm_accuracy
history_loaded_df['baseline_visible_precision'] = baseline_visible_aneurysm_precision
history_loaded_df['baseline_visible_recall'] = baseline_visible_aneurysm_recall

history_loaded_df['baseline_in_brain_accuracy'] = baseline_in_brain_accuracy
history_loaded_df['baseline_in_brain_precision'] = baseline_in_brain_precision
history_loaded_df['baseline_in_brain_recall'] = baseline_in_brain_recall

In [24]:
baseline_visible_aneurysm_accuracy, baseline_in_brain_accuracy

(np.float64(0.9007633587786259), np.float64(0.5155607751027598))

In [33]:
val_visible_metrics_df = history_loaded_df.melt(id_vars='epoch', value_vars=[
    'val_aneurysm_visible_in_image_accuracy_.5_visible', 'val_aneurysm_visible_in_image_precision_.5_visible', 'val_aneurysm_visible_in_image_recall_.5_visible',
    'baseline_visible_accuracy', 'baseline_visible_precision', 'baseline_visible_recall'
    ],
                                         var_name='metric', value_name='value')
px.line(val_visible_metrics_df, x='epoch', y='value', color="metric", markers=True,
        title='Visible Aneurysm Accuracy/Precision/Recall (Validation Set) Over Training History')

In [34]:
val_in_brain_metrics_df = history_loaded_df.melt(id_vars='epoch', value_vars=[
    'val_in_brain_with_aneurysm_accuracy_.5_in_brain', 'val_in_brain_with_aneurysm_precision_.5_in_brain', 'val_in_brain_with_aneurysm_recall_.5_in_brain',
    'baseline_in_brain_accuracy', 'baseline_in_brain_precision', 'baseline_in_brain_recall'
    ],
                                         var_name='metric', value_name='value')
px.line(val_in_brain_metrics_df, x='epoch', y='value', color="metric", markers=True,
        title='Aneurysm In Brain Accuracy/Precision/Recall (Validation Set) Over Training History')

In [35]:
loss_df = history_loaded_df.melt(id_vars='epoch', value_vars=['loss', 'val_loss', 'aneurysm_visible_in_image_loss', 'val_aneurysm_visible_in_image_loss', 'in_brain_with_aneurysm_loss', 'val_in_brain_with_aneurysm_loss'
    ],
                                         var_name='metric', value_name='value')
px.line(loss_df, x='epoch', y='value', color="metric", markers=True,
        title='Loss (Training and Validation Sets) Over Training History')